# Speakato sandbox - free model testing

In [1]:
import spacy
import pandas as pd
import numpy as np
import json
import keras.models

## Loading dataset and models

In [2]:
model_path = "../models/Speakato_model_2022-01-16"
dataset_path = "../examples/polish_commands_dataset"

with open(f"{model_path}/info.json", "r") as f:
    config = json.load(f)

with open(f"{dataset_path}/commands.txt", "r") as f:
    labels = [x.rstrip("\n") for x in f.readlines()]

config

{'language': 'pl',
 'spacy_model': 'pl_core_news_sm',
 'token_len': 96,
 'train_evaluation': [0.00011005385022144765, 1.0],
 'test_evaluation': [0.9265825152397156, 0.800000011920929]}

In [3]:
nlp = spacy.load(config["spacy_model"])
data = pd.read_json(f"{dataset_path}/dataset.json")
data

,text,command
0,Dzień dobry,greeting
1,Dobry wieczór,greeting
2,Cześć,greeting
3,Hej,greeting
4,Siema,greeting
5,"Elo, co tam",greeting
6,Witaj,greeting
7,Witam,greeting
8,Czołem,greeting
9,Joł,greeting


In [4]:
model = keras.models.load_model(f"{model_path}/model")

## Evaluation and comparison of the results

In [5]:
X_raw = data["text"]
X = np.array([nlp(x).vector for x in X_raw])
y = pd.get_dummies(data[["command"]]).values

model.evaluate(X, y)

2/2 [==============================] - 0s 4ms/step - loss: 2.5875 - accuracy: 0.4200


[2.5874805450439453, 0.41999998688697815]

In [6]:
y

array([[0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [1,

In [7]:
y_predicted = model(X)

In [8]:
for i, x in enumerate(X_raw):
    correct_index = y[i].tolist().index(max(y[i]))
    recognized_index = y_predicted[i].numpy().tolist().index(max(y_predicted[i]))
    print(f"{x}\nRecognized: {recognized_index}\nCorrect: {correct_index}\n")

Dzień dobry
Recognized: 4
Correct: 4

Dobry wieczór
Recognized: 4
Correct: 4

Cześć
Recognized: 4
Correct: 4

Hej
Recognized: 4
Correct: 4

Siema
Recognized: 4
Correct: 4

Elo, co tam
Recognized: 4
Correct: 4

Witaj
Recognized: 4
Correct: 4

Witam
Recognized: 1
Correct: 4

Czołem
Recognized: 4
Correct: 4

Joł
Recognized: 0
Correct: 4

Jak się masz
Recognized: 4
Correct: 4

Otwórz aplikację spotify
Recognized: 5
Correct: 5

Uruchom grę
Recognized: 5
Correct: 5

Otwórz menadżera zadań
Recognized: 5
Correct: 5

Uruchom wiersz poleceń
Recognized: 0
Correct: 5

Włącz chrome
Recognized: 4
Correct: 5

Otwórz notatnik
Recognized: 5
Correct: 5

Włącz mi aplikacje notatnik
Recognized: 5
Correct: 5

Uruchom przeglądarkę
Recognized: 7
Correct: 5

Uruchom kalkulator
Recognized: 5
Correct: 5

Odpal kalkulator
Recognized: 7
Correct: 5

Otwórz panel sterowania
Recognized: 0
Correct: 5

Otwórz ustawienia systemu
Recognized: 0
Correct: 5

Otwórz galerię zdjęć
Recognized: 1
Correct: 5

Zamknij okno
Recog

In [21]:
Predict = np.array([nlp(x).vector for x in ["Hej"]])
y_predicted = model(Predict)
recognized_index = y_predicted[0].numpy().tolist().index(max(y_predicted[0]))
print(recognized_index)

4
